In [9]:

from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "movie": "IronMan",
        "answer": """
        Let me introuce you about IronMan:
        Director: John Favreau
        Main Cast: Robert Downey Jr., Gwyneth Paltrow, Jeff Bridges
        Budget: $140 million
        Box Office: $585.2 million
        Genres: Action, Adventure, Sci-Fi
        Brief Synopsis: The film follows Tony Stark, a billionaire industrialist and genius inventor, who is kidnapped 
        """,
    },
    {
        "movie": "Avengers",
        "answer": """
        Let me introuce you about Avengers:
        Director: Joss Whedon
        Main Cast: Robert Downey Jr., Chris Evans, Scarlett Johansson
        Budget: $220 million
        Box Office: $1.519 billion
        Genres: Action, Adventure, Sci-Fi
        Brief Synopsis: The film follows Nick Fury, director of the peacekeeping organization S.H.I.E.L.D., who recruits Iron
        """,
    }
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {movie}?"),
        ("ai", "{answer}")
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {movie}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"movie": "Argylle"})

"Argylle" is an upcoming spy thriller film directed by Matthew Vaughn. The film stars Henry Cavill, Sam Rockwell, Bryce Dallas Howard, and Dua Lipa. It is based on the upcoming novel of the same name by Ellie Conway.

AIMessageChunk(content='"Argylle" is an upcoming spy thriller film directed by Matthew Vaughn. The film stars Henry Cavill, Sam Rockwell, Bryce Dallas Howard, and Dua Lipa. It is based on the upcoming novel of the same name by Ellie Conway.')